In [ ]:
import torch
import torchaudio

from torch.optim import AdamW
from torch.utils.data import DataLoader

from torch import nn

import pandas as pd
import numpy as np

from tqdm.auto import tqdm

from sklearn.model_selection import KFold, StratifiedKFold

from transformers import PreTrainedTokenizerFast

from transformers import get_cosine_schedule_with_warmup

from modules.Transformer.model import Transformer
from modules.Transformer.train import train_epoch, eval_epoch
from modules.dataset import AudioDataset

import random
import os


import wandb

torch.cuda.is_available()

In [ ]:
path_to_tokenizer = './tokenizer.json'
path_to_data = './audio_dataset/'

data = pd.read_csv(os.path.join(path_to_data,'df.csv'), usecols=['text','status','path','rate','duration','frames'])
data = data[data.status=='APPROVED'].reset_index(drop=True)
del data['status']
data.text = data.text.apply(lambda x: "".join([char for char in x if char.isalpha() or char==' ']).lower())
data.duration.max()

train_data = data.iloc[:10000]
valid_data = data.iloc[70000:]

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.displot(data.text.str.len())
# plt.show()dd
# sns.displot(data.duration)
# plt.show()

In [ ]:
np.percentile(data.text.str.len(), 99.5)

In [ ]:
def weights_init_xavier(m):
    '''
    Xavier uniform
    '''
    classname = m.__class__.__name__

    if classname.find('Linear') != -1:
        torch.nn.init.xavier_uniform_(m.weight)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = PreTrainedTokenizerFast(tokenizer_file=path_to_tokenizer, 
                                    padding_side ='right',
                                    bos_token = '[SOS]',
                                    eos_token = '[EOS]',
                                    pad_token = '[PAD]',
                                    unk_token = '[UNK]',
                                    mask_token = '[MASK]')


train_dataset = AudioDataset(train_data, path_to_data, tokenizer, 
                             n_fft=512,
                             n_mels=128, 
                             center=True, 
                             max_tokenized_length=100, 
                             max_audio_len=25, 
                             sr=16000)
valid_dataset = AudioDataset(valid_data, path_to_data, tokenizer, 
                             n_fft=512, 
                             n_mels=128, 
                             center=True, 
                             max_tokenized_length=100, 
                             max_audio_len=25, 
                             sr=16000)
model = Transformer(vocab_size=len(tokenizer),
                    n_mels=128,
                    enc_seq_len=25, 
                    dec_seq_len=100,
                    hidden_dim=16, 
                    enc_num_layers=2, 
                    dec_num_layers=2, 
                    num_heads=3, 
                    ff_dim=128, 
                    device=device,
                    dropout=0.0, 
                    sr=16000, 
                    n_fft=512,
                    padding_idx=tokenizer.pad_token_id)


        #.bias.data.fill_(0.01)
model.apply(weights_init_xavier)

n_fft=1024, win_lenght=1024, hop_lenght=256, n_mels=64, center=True):

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from torchmetrics.functional import word_error_rate
from torchmetrics.functional.classification import multiclass_accuracy

In [ ]:
def cross_validation(model, 
                     dataset, 
                     loss_function,
                     strat_array=None,
                     device=torch.device("cuda"),
                     random_state: int=69, 
                     shuffle: bool=True, 
                     n_folds: int=10, 
                     epochs: int=5, 
                     lr: float=1e-3,
                     start_fold: int=0, 
                     batch_size: int=4,
                     iters_to_accumulate=None,
                     n_accumulated_grads: int = 0):
    random.seed(random_state),
    np.random.seed(random_state)
    torch.manual_seed(random_state)
    torch.cuda.manual_seed_all(random_state)
    
    loss_function.to(device)
    if strat_array:
        kfold = StratifiedKFold(n_folds, shuffle=shuffle, random_state=random_state)
        split = kfold.split(dataset, strat_array)
    else: 
        kfold = KFold(n_folds, shuffle=shuffle, random_state=random_state)
        split = kfold.split(dataset)

    for fold, (train_ids, eval_ids) in enumerate(split):
        if fold >= start_fold:
            print(f'FOLD {fold}')
            print('--------------------------------')
            run = wandb.init(
                name=f"fold_{fold}",
                project=f"asr_fold_{fold}",
                config={ 
                         "random_state": random_state, 
                         "shuffle": shuffle,
                         "epochs": epochs, 
                         "learning_rate": lr,
                         "batch_size": batch_size,
                         "iters_to_accumulate": iters_to_accumulate
                        })

            optimizer = AdamW(model.parameters(), lr=lr
#                 [{"params": model.encoder.parameters(), "lr": 1e-4},
#                 {"params": model.decoder.parameters(), "lr": 1e-3},]
        )

            train_subsampler = torch.utils.data.Subset(dataset,  train_ids)
            train_loader = torch.utils.data.DataLoader(
                          train_subsampler, 
                          batch_size=batch_size,
                          shuffle=shuffle, drop_last=True)

            eval_subsampler = torch.utils.data.Subset(dataset,  eval_ids)
            eval_loader = torch.utils.data.DataLoader(
                          eval_subsampler,
                          batch_size=batch_size,
                          shuffle=shuffle, drop_last=True)
            
            total_steps = len(train_loader) * epochs 

            scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                                    num_warmup_steps = 0, # Default value in run_glue.py
                                                    num_training_steps = total_steps)


            for epoch_i in range(epochs):
                train_metrics, t_preds = train_epoch(model, train_loader, dataset.tokenizer, loss_function, optimizer, scheduler, device)
                eval_metrics, preds = eval_epoch(model, eval_loader, dataset.tokenizer, loss_function, device)
                print(f"EPOCH: {epoch_i}")
                print(train_metrics)
                print(eval_metrics)
                print(t_preds)
                print(preds)
                run.log(train_metrics)
                run.log(eval_metrics)
                            
            run.finish()
        break

In [ ]:
string = tokenizer.encode("я люблю дашу")
tokenizer.decode(string, skip_special_tokens=True)

In [ ]:
tokenizer.decode(16)

In [ ]:
train_dataset[0]['encoded_text']

In [ ]:
train_dataset[0]['true_text']

In [ ]:
cross_validation(model = model,
                 dataset=train_dataset, 
                 loss_function=nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id, reduction="mean"), 
                 device=torch.device("cuda"),
                 random_state=69,
                 shuffle=True,
                 batch_size=16)